In [ ]:
import torch as th
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def task_df(vfs_filepath, gt_filepath, zone_filepath, task_name):
    vfs_robs = th.load(vfs_filepath)
    gt_robs = th.load(gt_filepath)
    zone_gt = th.load(zone_filepath)

    device = th.device('cpu')
    vfs_robs = vfs_robs.flatten().to(device).numpy()
    gt_robs = gt_robs.flatten().to(device).numpy()
    zone_gt = np.array(zone_gt)

    task_names = np.repeat(task_name, len(zone_gt))

    combined = np.stack((vfs_robs, gt_robs, zone_gt, task_names), axis=1)
    column_vales = ['VFS Robustness', 'State Space Robustness', 'zone_gt', 'task']

    df = pd.DataFrame(data=combined, columns=column_vales)
    return df


In [ ]:
# construct dataframe with [vfs_robs, gt_robs, zone_gt]
chain_vfs_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/vfs_robs__mcts_mpc_chain_3c3_20240321_162656.pt'
chain_gt_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/ground_truth_mcts_mpc_chain_3c3_20240321_162656.pt'
chain_zone_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/zone_truth__mcts_mpc_chain_3c3_20240321_162656.pt'

df_chain = task_df(chain_vfs_path, chain_gt_path, chain_zone_path, 'chain')

avoid_vfs_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/vfs_robs_mcts_mpc_avoid_20240322_091543.pt'
avoid_gt_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/ground_truth_mcts_mpc_avoid_20240322_091543.pt'
avoid_zone_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/zone_truth_mcts_mpc_avoid_20240322_091543.pt'

df_avoid = task_df(avoid_vfs_path, avoid_gt_path, avoid_zone_path, 'avoid')

stable_vfs_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/ground_truth_mcts_mpc_stable_20240322_115317.pt'
stable_gt_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/ground_truth_mcts_mpc_stable_20240322_115317.pt'
stable_zone_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/zone_truth_mcts_mpc_stable_20240322_115317.pt'

df_stable = task_df(avoid_vfs_path, avoid_gt_path, avoid_zone_path, 'stable')

df_plot = pd.concat([df_chain, df_avoid, df_stable], axis=0)
df_plot['VFS Robustness'] = pd.to_numeric(df_plot['VFS Robustness'])
df_plot['State Space Robustness'] = pd.to_numeric(df_plot['State Space Robustness'])

In [ ]:
mdf = pd.melt(df_plot[['VFS Robustness', 'State Space Robustness', 'task']], id_vars=['task'])
mdf

In [ ]:
fig, ax = plt.subplots()
color_dic = {
    'VFS Robustness': 'yellow', 
    'State Space Robustness': 'red'
}
sns.boxplot(data=mdf, x="task", y="value", hue='variable', ax=ax, palette=color_dic, gap=.1)
# ax.set_ylim(-0.5, 0.5)
plt.show()
fig.savefig('time_step=100.png')